In [2]:
!pip install selenium
!pip install bs4

  Using cached selenium-4.33.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
Using cached selenium-4.33.0-py3-none-any.whl (9.4 MB)
Using cached certifi-2025.4.26-py3-none-any.whl (159 kB)
Using cached trio-0.30.0-py3-none-any.whl (499 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached typing_


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
Using cached soupsieve-2.7-py3-none-any.whl (36 kB)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import time
import json

def parse_scholarship_detail(soup: BeautifulSoup) -> dict:
    def get_text_after_heading(heading_text):
        heading = soup.find(lambda tag: tag.name == "h2" and heading_text in tag.text)
        if not heading:
            return None
        next_tag = heading.find_next_sibling()
        if not next_tag:
            return None
        if next_tag.name == "p":
            return next_tag.get_text(strip=True)
        elif next_tag.name in ["ol", "ul"]:
            return [li.get_text(strip=True) for li in next_tag.find_all("li")]
        return None

    data = {}
    data["ทุนการศึกษา"] = get_text_after_heading("ชื่อทุนการศึกษา")
    data["หน่วยงานให้ทุน"] = get_text_after_heading("หน่วยงานให้ทุนการศึกษา")
    data["คำอธิบาย"] = get_text_after_heading("คำอธิบาย")
    data["คุณสมบัติผู้รับทุน"] = get_text_after_heading("คุณสมบัติผู้รับทุน")
    data["การสนับสนุนด้านทุนการศึกษา"] = get_text_after_heading("การสนับสนุนด้านทุนการศึกษา")
    data["ขั้นตอนการขอรับทุน"] = get_text_after_heading("ขั้นตอนการขอรับทุน")
    data["วันเปิดรับสมัคร"] = get_text_after_heading("วันเปิดรับสมัคร")

    # ดึงข้อมูลติดต่อแหล่งทุน (ลิงก์, เบอร์โทร ฯลฯ)
    contact_heading = soup.find(lambda tag: tag.name == "h2" and "ติดต่อแหล่งทุน" in tag.text)
    contact_info = []
    if contact_heading:
        next_tag = contact_heading.find_next_sibling()
        while next_tag and next_tag.name in ["p", "a", "h2"]:
            # ดึงลิงก์
            if next_tag.name == "a":
                href = next_tag.get("href")
                if href:
                    contact_info.append(href)
            # ดึงลิงก์ใน p
            elif next_tag.name == "p":
                links = next_tag.find_all("a")
                for a in links:
                    href = a.get("href")
                    if href:
                        contact_info.append(href)
                # ดึงข้อความใน p (เช่นเบอร์โทร)
                text = next_tag.get_text(strip=True)
                if text:
                    contact_info.append(text)
            next_tag = next_tag.find_next_sibling()
    data["ติดต่อแหล่งทุน"] = contact_info if contact_info else None

    return data

In [13]:
# ตั้งค่า options สำหรับ Chrome
options = Options()
options.add_argument('--headless')  # รัน Chrome แบบไม่แสดงหน้าต่าง
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

# สร้าง instance ของ Chrome WebDriver
driver = webdriver.Chrome(options=options)

try:
    # เปิดหน้าเว็บ list ทุน
    url = 'https://findstudentship.eef.or.th/scholarship?grade=ทุกระดับ&cost=ทุนทั้งหมด&genre=ทุนให้เปล่า'
    driver.get(url)

    # รอให้หน้าเว็บโหลด
    time.sleep(3)

    # # คลิก "ดูเพิ่มเติม" ซ้ำๆ จนกว่าจะไม่มีให้คลิก
    # click_next_limit = 1
    # click_times = 0
    # while True:
    #     try:
    #         load_more_button = WebDriverWait(driver, 5).until(
    #             EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'ดูเพิ่มเติม')]"))
    #         )
    #         driver.execute_script("arguments[0].click();", load_more_button)
    #         time.sleep(2)
    #         click_times += 1
    #         print(f"Clicked 'ดูเพิ่มเติม' {click_times} times")
    #     except (TimeoutException, ElementClickInterceptedException):
    #         break
    #     if click_times >= click_next_limit:
    #         print("Reached click limit, stopping.")
    #         break

    # รอให้เนื้อหาหลังโหลดเพิ่มเติมเสร็จ
    time.sleep(2)

    # เก็บ list ของลิงก์ 'ไปยังแหล่งทุน'
    Datas = []

    while True:
        link_elements = driver.find_elements(By.XPATH, "//div[contains(text(),'ไปยังแหล่งทุน')]")
        total = len(link_elements)
        print(f"พบทั้งหมด {total} ทุน")

        for i in range(len(Datas), total):
            try:
                link_elements = driver.find_elements(By.XPATH, "//div[contains(text(),'ไปยังแหล่งทุน')]")
                driver.execute_script("arguments[0].click();", link_elements[i])
                time.sleep(5)

                # ดึงหน้า detail แล้วเก็บ <h1>
                detail_html = driver.page_source
                detail_soup = BeautifulSoup(detail_html, 'html.parser')
                data = parse_scholarship_detail(detail_soup)

                Datas.append(data)

                driver.back()  # กลับไปยังหน้า list
                time.sleep(3)
                print(len(Datas), "ทุนที่ดึงข้อมูลแล้ว")

            except IndexError:
                print("องค์ประกอบหายไปจาก DOM หลัง back กลับมา รีโหลดลูป")
                break  # ออกจากลูปในกรณีองค์ประกอบหาย
        else:
            break  # ถ้าลูปทำจบครบทุกตัวโดยไม่ break → ออกลูป while

    print("ผลลัพธ์:")
    print(Datas)

finally:
    driver.quit()


พบทั้งหมด 10 ทุน
1 ทุนที่ดึงข้อมูลแล้ว
2 ทุนที่ดึงข้อมูลแล้ว
3 ทุนที่ดึงข้อมูลแล้ว
4 ทุนที่ดึงข้อมูลแล้ว
5 ทุนที่ดึงข้อมูลแล้ว
6 ทุนที่ดึงข้อมูลแล้ว
7 ทุนที่ดึงข้อมูลแล้ว
8 ทุนที่ดึงข้อมูลแล้ว
9 ทุนที่ดึงข้อมูลแล้ว
10 ทุนที่ดึงข้อมูลแล้ว
ผลลัพธ์:
[{'ทุนการศึกษา': 'ทุนการศึกษาพระราชทาน “นวฤกษ์”', 'หน่วยงานให้ทุน': 'มูลนิธิช่วยนักเรียนที่ขาดแคลน ในพระบรมราชินูปถัมภ์ (ม.น.ข.)', 'คำอธิบาย': 'ทุนนวฤกษ์ ทุนการศึกษาพระราชทานนี้ มีความมุ่งหมายที่จะช่วยเหลือนักเรียนที่เป็นคนดี และมีความประพฤติดี แต่ขาดทุนทรัพย์ให้ได้ศึกษาต่อภายในประเทศจนกว่าจำสำเร็จการศึกษาในระดับการศึกษาตามสมควรแก่อัตภาพ เพื่อช่วยเหลือนักเรียนที่มีความประพฤติดี  ขยัน  อดทน แต่ขาดแคลนทุนทรัพย์ให้มีโอกาสได้ศึกษาต่อ สายอาชีพ ระดับ ปวช. ปวส. และระดับปริญญาตรีตามสมควรแก่ศักยภาพของผู้เรียนโดยไม่มีข้อผูกมัดใด ๆ', 'คุณสมบัติผู้รับทุน': ['เป็นนักเรียน ซึ่งได้รับการคัดเลือกขั้นต้นจากสถานศึกษาของผู้สมัครขอรับทุน', 'กำลังเรียนอยู่ชั้นมัธยมศึกษาปีที่ 6 มีผลการเรียนในปีที่ผ่านมาไม่ต่ำกว่า 2.50', 'มีฐานะยากจน  สถานศึกษาเห็นว่าสมควรได้รับความ

In [14]:
with open("scholarship_detail.json", 'w', encoding='utf-8') as f:
    json.dump(Datas, f, ensure_ascii=False, indent=4)